# LumaFin - Complete Pipeline Evaluation

This notebook evaluates the complete LumaFin categorization pipeline.

**What this notebook does:**
1. Loads all trained models (embeddings, FAISS, reranker)
2. Runs comprehensive evaluation on test set
3. Compares baseline vs fine-tuned performance
4. Generates detailed accuracy reports
5. Creates visualizations of results

**Runtime:** GPU helpful for embeddings, but CPU OK
**Time:** ~10-20 minutes

## Step 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q sentence-transformers xgboost faiss-cpu pandas numpy scikit-learn matplotlib seaborn tqdm

## Step 2: Load Models and Data

In [ ]:
import pandas as pd
import numpy as np
import pickle
import faiss
from sentence_transformers import SentenceTransformer
import os

# Load test data
test_file = '/content/drive/MyDrive/LumaFin/data/test.csv'
df_test = pd.read_csv(test_file)
print(f"✅ Test set: {len(df_test)} examples")
print(f"\nCategory distribution:")
print(df_test['category'].value_counts())

In [ ]:
# Load embedding models
base_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
finetuned_path = '/content/drive/MyDrive/LumaFin/models/lumafin-lacft-v1.0'

print("Loading embedding models...")
base_model = SentenceTransformer(base_model_name)
print(f"✅ Base model loaded")

if os.path.exists(finetuned_path):
    finetuned_model = SentenceTransformer(finetuned_path)
    print(f"✅ Fine-tuned model loaded")
else:
    print(f"⚠️ Fine-tuned model not found, will use base model only")
    finetuned_model = base_model

In [ ]:
# Load FAISS index and metadata
faiss_path = '/content/drive/MyDrive/LumaFin/models/faiss_index.bin'
metadata_path = '/content/drive/MyDrive/LumaFin/models/faiss_metadata.pkl'

if os.path.exists(faiss_path) and os.path.exists(metadata_path):
    index = faiss.read_index(faiss_path)
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
    
    train_categories = metadata['categories']
    all_categories = metadata['all_categories']
    
    print(f"✅ FAISS index loaded: {index.ntotal} vectors")
    print(f"✅ Categories: {all_categories}")
else:
    print("❌ FAISS index not found. Please run notebook 03 first.")
    raise FileNotFoundError("FAISS index required")

In [ ]:
# Load reranker model
reranker_path = '/content/drive/MyDrive/LumaFin/models/xgb_reranker.pkl'

if os.path.exists(reranker_path):
    with open(reranker_path, 'rb') as f:
        reranker_model = pickle.load(f)
    print(f"✅ Reranker model loaded")
else:
    print("⚠️ Reranker not found. Will evaluate retrieval only.")
    reranker_model = None

## Step 3: Define Evaluation Functions

In [ ]:
from collections import Counter
from tqdm import tqdm

def create_text(row):
    desc = row.get('description', '')
    return f"{row['merchant']} {desc} ${row['amount']:.2f}"

def retrieve_top_k(query_text, model, index, k=20):
    """Retrieve top-k candidates using FAISS."""
    query_emb = model.encode([query_text])[0].astype('float32')
    query_emb = query_emb / np.linalg.norm(query_emb)
    
    scores, indices = index.search(np.array([query_emb]), k)
    candidates = [(train_categories[i], scores[0][j]) for j, i in enumerate(indices[0])]
    
    return candidates

def predict_by_voting(candidates):
    """Predict category by majority voting."""
    votes = Counter([cat for cat, _ in candidates])
    return votes.most_common(1)[0][0]

def predict_by_reranker(candidates, all_categories, reranker_model):
    """Predict category using reranker."""
    # Aggregate scores by category
    category_scores = {cat: [] for cat in all_categories}
    for cat, score in candidates:
        category_scores[cat].append(score)
    
    # Extract features
    features = []
    for cat in all_categories:
        scores = category_scores[cat]
        feat = [
            len(scores),
            sum(scores) if scores else 0,
            max(scores) if scores else 0,
            np.mean(scores) if scores else 0,
            min(scores) if scores else 0,
            len(scores) / len(candidates) if candidates else 0,
            0
        ]
        features.append(feat)
    
    X = np.array(features)
    probs = reranker_model.predict_proba(X)[:, 1]
    pred_idx = np.argmax(probs)
    
    return all_categories[pred_idx], probs[pred_idx]

print("✅ Evaluation functions defined")

## Step 4: Evaluate Retrieval (Baseline)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Evaluating retrieval baseline (base model + voting)...\n")

predictions_baseline = []
true_labels = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Baseline"):
    query_text = create_text(row)
    candidates = retrieve_top_k(query_text, base_model, index, k=20)
    pred = predict_by_voting(candidates)
    
    predictions_baseline.append(pred)
    true_labels.append(row['category'])

accuracy_baseline = accuracy_score(true_labels, predictions_baseline)
print(f"\n✅ Baseline Accuracy: {accuracy_baseline:.1%}")
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, predictions_baseline, zero_division=0))

## Step 5: Evaluate Fine-Tuned Model

In [ ]:
print("Evaluating fine-tuned model + voting...\n")

predictions_finetuned = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Fine-tuned"):
    query_text = create_text(row)
    candidates = retrieve_top_k(query_text, finetuned_model, index, k=20)
    pred = predict_by_voting(candidates)
    
    predictions_finetuned.append(pred)

accuracy_finetuned = accuracy_score(true_labels, predictions_finetuned)
print(f"\n✅ Fine-tuned Accuracy: {accuracy_finetuned:.1%}")
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, predictions_finetuned, zero_division=0))

## Step 6: Evaluate with Reranker

In [ ]:
if reranker_model:
    print("Evaluating fine-tuned model + reranker...\n")
    
    predictions_reranker = []
    confidences_reranker = []
    
    for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Reranker"):
        query_text = create_text(row)
        candidates = retrieve_top_k(query_text, finetuned_model, index, k=20)
        pred, conf = predict_by_reranker(candidates, all_categories, reranker_model)
        
        predictions_reranker.append(pred)
        confidences_reranker.append(conf)
    
    accuracy_reranker = accuracy_score(true_labels, predictions_reranker)
    print(f"\n✅ Reranker Accuracy: {accuracy_reranker:.1%}")
    print(f"   Average Confidence: {np.mean(confidences_reranker):.3f}")
    print(f"\n📊 Classification Report:")
    print(classification_report(true_labels, predictions_reranker, zero_division=0))
else:
    print("⚠️ Skipping reranker evaluation (model not loaded)")
    predictions_reranker = None
    accuracy_reranker = None

## Step 7: Comparison and Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Accuracy comparison
results = {
    'Baseline\n(Base + Voting)': accuracy_baseline,
    'Fine-tuned\n(L-A CFT + Voting)': accuracy_finetuned,
}

if accuracy_reranker is not None:
    results['Complete\n(Fine-tuned + Reranker)'] = accuracy_reranker

# Plot
plt.figure(figsize=(10, 6))
bars = plt.bar(results.keys(), [v * 100 for v in results.values()], 
               color=['#3498db', '#2ecc71', '#e74c3c'])
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('LumaFin Pipeline Performance Comparison', fontsize=14, fontweight='bold')
plt.ylim(0, 100)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/LumaFin/results_comparison.png', dpi=150)
plt.show()

print("✅ Comparison chart saved")

In [ ]:
# Confusion matrix for best model
if predictions_reranker:
    best_predictions = predictions_reranker
    model_name = "Complete Pipeline"
else:
    best_predictions = predictions_finetuned
    model_name = "Fine-tuned Model"

cm = confusion_matrix(true_labels, best_predictions, labels=all_categories)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=all_categories, yticklabels=all_categories)
plt.title(f'Confusion Matrix - {model_name}', fontsize=14, fontweight='bold')
plt.ylabel('True Category', fontsize=12)
plt.xlabel('Predicted Category', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/LumaFin/confusion_matrix.png', dpi=150)
plt.show()

print("✅ Confusion matrix saved")

## Step 8: Per-Category Analysis

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate per-category metrics
precision, recall, f1, support = precision_recall_fscore_support(
    true_labels, best_predictions, labels=all_categories, zero_division=0
)

# Create DataFrame
metrics_df = pd.DataFrame({
    'Category': all_categories,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

metrics_df = metrics_df.sort_values('F1-Score', ascending=False)

print("\n📊 Per-Category Performance:")
print(metrics_df.to_string(index=False))

# Save to CSV
metrics_df.to_csv('/content/drive/MyDrive/LumaFin/per_category_metrics.csv', index=False)
print("\n✅ Metrics saved to per_category_metrics.csv")

In [ ]:
# Visualize per-category F1 scores
plt.figure(figsize=(12, 6))
bars = plt.bar(metrics_df['Category'], metrics_df['F1-Score'], color='#3498db')
plt.xlabel('Category', fontsize=12)
plt.ylabel('F1 Score', fontsize=12)
plt.title('Per-Category F1 Scores', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)

# Add horizontal line for average
avg_f1 = metrics_df['F1-Score'].mean()
plt.axhline(y=avg_f1, color='r', linestyle='--', label=f'Average: {avg_f1:.3f}')
plt.legend()

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/LumaFin/per_category_f1.png', dpi=150)
plt.show()

print("✅ Per-category chart saved")

## Step 9: Generate Final Report

In [ ]:
# Generate summary report
report = f"""
# LumaFin Evaluation Report

## Dataset
- Test Examples: {len(df_test)}
- Categories: {len(all_categories)}
- Training Index Size: {index.ntotal} vectors

## Performance Summary

### Model Accuracy
| Model | Accuracy |
|-------|----------|
| Baseline (Base + Voting) | {accuracy_baseline:.1%} |
| Fine-tuned (L-A CFT + Voting) | {accuracy_finetuned:.1%} |
"""

if accuracy_reranker:
    report += f"| Complete (Fine-tuned + Reranker) | {accuracy_reranker:.1%} |\n"

report += f"""

### Improvement
- Fine-tuning gain: {(accuracy_finetuned - accuracy_baseline)*100:+.1f}%
"""

if accuracy_reranker:
    report += f"- Reranker gain: {(accuracy_reranker - accuracy_finetuned)*100:+.1f}%\n"
    report += f"- Total improvement: {(accuracy_reranker - accuracy_baseline)*100:+.1f}%\n"

report += f"""

### Per-Category Performance
- Average F1-Score: {metrics_df['F1-Score'].mean():.3f}
- Best Category: {metrics_df.iloc[0]['Category']} (F1={metrics_df.iloc[0]['F1-Score']:.3f})
- Worst Category: {metrics_df.iloc[-1]['Category']} (F1={metrics_df.iloc[-1]['F1-Score']:.3f})

## Files Generated
- `results_comparison.png` - Accuracy comparison chart
- `confusion_matrix.png` - Confusion matrix heatmap
- `per_category_f1.png` - Per-category F1 scores
- `per_category_metrics.csv` - Detailed metrics table

## Conclusion
The LumaFin pipeline achieves **{max(accuracy_baseline, accuracy_finetuned, accuracy_reranker or 0):.1%} accuracy** on the test set.
"""

if accuracy_reranker and accuracy_reranker >= 0.90:
    report += "\n✅ **TARGET ACHIEVED: >90% accuracy**"

# Save report
with open('/content/drive/MyDrive/LumaFin/evaluation_report.md', 'w') as f:
    f.write(report)

print(report)
print("\n✅ Report saved to evaluation_report.md")

## ✅ Evaluation Complete!

All results have been saved to your Google Drive at:
```
/content/drive/MyDrive/LumaFin/
```

### Generated Files:
- 📊 `evaluation_report.md` - Complete evaluation summary
- 📈 `results_comparison.png` - Model comparison chart
- 🎯 `confusion_matrix.png` - Confusion matrix
- 📉 `per_category_f1.png` - Category performance
- 📋 `per_category_metrics.csv` - Detailed metrics

### Next Steps for Hackathon:
1. Download all trained models and results from Google Drive
2. Integrate into your local repository
3. Prepare demo presentation
4. Test the API and Streamlit UI
5. Create demo video showcasing the system